In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt

from matplotlib import pyplot as plt
from datetime import timedelta


In [2]:
mobile_dataset = pd.read_csv('mobile_dataset.csv')

In [3]:
mobile_dataset = mobile_dataset.rename(columns = {'event.time':'event_time', 'event.name': 'event_name', 'user.id':'user_id'})
mobile_dataset.head(1)

,event_time,event_name,user_id
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894


In [4]:
mobile_dataset['event_time'] = pd.to_datetime(mobile_dataset['event_time'])
mobile_dataset['event_time']  = mobile_dataset['event_time'].dt.floor('1S')

In [5]:
def my_recode(name):
    if 'search' in name:
        return 'search'
    else:
        return name

In [6]:
mobile_dataset['event_name'] = mobile_dataset['event_name'].apply(my_recode)
mobile_dataset['event_name'].value_counts()

tips_show        40055
photos_show      10012
search            6784
advert_open       6164
contacts_show     4450
map               3881
favorites_add     1417
tips_click         814
contacts_call      541
show_contacts       79
Name: event_name, dtype: int64

In [7]:
def my_recode2(name):
    if 'contact' in name:
        return 'contacts_show'
    else:
        return name

In [8]:
mobile_dataset['event_name'] = mobile_dataset['event_name'].apply(my_recode2)
mobile_dataset['event_name'].value_counts()

tips_show        40055
photos_show      10012
search            6784
advert_open       6164
contacts_show     5070
map               3881
favorites_add     1417
tips_click         814
Name: event_name, dtype: int64

In [14]:
mobile_sources = pd.read_csv('mobile_soures.csv')


In [15]:
mobile_sources = mobile_sources.rename(columns = {'userId':'user_id'})

In [16]:
data = mobile_dataset.merge(mobile_sources, on='user_id')

In [17]:
# удалим дубликаты
data = data.drop_duplicates()
data.duplicated().sum()

0

In [18]:
#identify difference 60Min for each group with cumulative sum
g = (data.groupby('user_id')['event_time'].diff() > pd.Timedelta('60Min')).cumsum()
g
data['session_id'] = data.groupby(['user_id', g], sort=False).ngroup() + 1

In [19]:
data['session_id'].nunique()

9573

In [20]:
data.to_csv(r'/home/shinshilla/Documents/data_final.csv', index = False) 

In [22]:
data['user_id'].nunique()

4293

In [23]:
mobile_dataset['event_time'].max()

Timestamp('2019-11-03 23:58:12')